#  Rozwiązywanie układu równań liniowych metodami rozkładu LU

Podaj zasadę działania metod opartych o dekompozycję LU.

## Zadanie 1

Zaimplementuj metody rozkładu LU:
- Rozkład Crouta 
- Rozkład Doolitla
- Rozkład Choleskyego

Dla każdej z metod podaj warunki niezbędne aby można ją było zastosować. Sprawdź poprawność działania tych metod.
Przetestuj wydajność algorytmów dla kilku rozmiarów macierzy (podobnie jak w ćwiczeniu 9)

**dekompozycja LU** - metoda rozwiązywania układu równań liniowych - nazwa metody wywodzi się od używanych w tej metodzie dwóch macierzy trójkątnych, tj. trójkątnej dolnej i trójkątnej górnej

**zasada działania:**
mając dany układ równań $A * x = y$, gdzie $A$ jest macierzą współczynników, $x$ - wektorem niewiadomych, a $y$ - wektorem danych, zapisujemy macierz współczynników jako iloczyn dóch macierzy trójkątnych $L$ i $U$ - $A = L * U$
wówczas układ równań przyjmuje postać $L * U * x = y$, a jego roziwązanie sprowadza się do rozwiązania dwóch układów równań:
$L * z = y$ oraz $U * x = z$

**metoda Crouta**
- diagonala macierzy $U$ wypełniona jest 1

In [0]:
def crout(A):
    n = len(A)
    L = [[0] * n for i in range(n)]
    U = [[0] * n for i in range(n)]

    for j in range(n):
        U[j][j] = 1
        for i in range(j, n):
            tmp_l = float(A[i][j])
            for k in range(j):
                tmp_l -= L[i][k] * U[k][j]
            L[i][j] = tmp_l
        for i in range(j + 1, n):
            tmp_u = float(A[j][i])
            for k in range(j):
                tmp_u -= L[j][k] * U[k][i]
            if int(L[j][j]) == 0:
                raise ZeroDivisionError('attempted to divide by 0')
            U[j][i] = tmp_u / L[j][j]
    return L, U


**metoda Doolittla**
- diagonala macierzy $L$ wypełniona jest 1

In [0]:
def doolittle(A):
    n = len(A)
    L = [[0] * n for i in range(n)]
    U = [[0] * n for i in range(n)]

    for i in range(n):
        L[i][i] = 1

        for k in range(i, n):
            sum = 0
            for j in range(i):
                sum += (L[i][j] * U[j][k])

            U[i][k] = A[i][k] - sum

        for k in range(i, n):
            sum = 0
            for j in range(i):
                sum += (L[k][j] * U[j][i])

            L[k][i] = (A[k][i] - sum) / U[i][i]

    return L, U

**metoda Choleskyego**
- macierz A musi być wypełniona liczbami rzeczywistymi
- macierz A musi być symetyczna, czyli $A^{T} = A$
- macierz A musi być oddatnio określona, czyli $x^{T}Ax > 0$ dla każdego wektora x

In [0]:
from math import sqrt
import numpy as np

def cholesky(A):
    n = len(A)
    L = [[0] * n for i in range(n)]

    for i in range(n):
        for k in range(i + 1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))

            if i == k:
                L[i][k] = sqrt(A[i][i] - tmp_sum)
            else:
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L, np.array(L).transpose()

**sprawdzenie poprawności** - sprawdzam działanie dla macierzy o rozmiarze 4x4, wypełnionej w taki sposób by spełniała wymagania dla zastosowania każdej z metod

In [0]:
def decompose_and_solve(A, Y, method):
    L, U = method(A)
    z = np.linalg.solve(L, Y)
    x = np.linalg.solve(U, z)
    return x

def random_matrix(min, max, cols, rows):
    return np.random.randint(low=min, high=max + 1, size=(rows, cols))

def symetric_positive_matrix(min, max, cols, rows):
    A = random_matrix(min, max, cols, rows)
    B = np.dot(A, A.transpose())
    return (B + B.T) / 2

def print_matrix(A):
    for i in A:
        print(i)

In [60]:
A = symetric_positive_matrix(4, 12, 4, 4)
Y = create_rand_vector(4, 12, 4)
L_crout, U_crout = crout(A)
L_doolittle, U_doolittle = doolittle(A)
L_cholesky, U_cholesky = cholesky(A)

A_crout = np.matmul(L_crout, U_crout)
A_doolittle = np.matmul(L_doolittle, U_doolittle)
A_cholesky = np.matmul(L_cholesky, U_cholesky)

print("macierz A:")
print_matrix(A)
print("\nrozkład Crouta:")
print("macierz L:")
print_matrix(L_crout)
print("macierz U:")
print_matrix(U_crout)
print("wektor x:")
print(decompose_and_solve(A, Y, crout))
print("L * U:")
print_matrix(A_crout)

print("\nrozkład Doolittla:")
print("macierz L:")
print_matrix(L_doolittle)
print("macierz U:")
print_matrix(U_doolittle)
print("wektor x:")
print(decompose_and_solve(A, Y, doolittle))
print("L * U:")
print_matrix(A_doolittle)

print("\nrozkład Choleskyego:")
print("macierz L:")
print_matrix(L_cholesky)
print("macierz U:")
print_matrix(U_cholesky)
print("wektor x")
print(decompose_and_solve(A, Y, cholesky))
print("L * U:")
print_matrix(A_cholesky)

macierz A:
[282. 220. 216. 286.]
[220. 233. 208. 237.]
[216. 208. 201. 226.]
[286. 237. 226. 319.]

rozkład Crouta:
macierz L:
[282.0, 0, 0, 0]
[220.0, 61.36879432624113, 0, 0]
[216.0, 39.48936170212767, 10.142725066450922, 0]
[286.0, 13.879432624113463, -1.9949150583612418, 25.41186122030423]
macierz U:
[1, 0.7801418439716312, 0.7659574468085106, 1.0141843971631206]
[0, 1, 0.6434762510112102, 0.2261643360684154]
[0, 0, 1, -0.19668432746539175]
[0, 0, 0, 1]
wektor x:
[-0.06660264  0.03129035  0.15618005 -0.05537351]
L * U:
[282. 220. 216. 286.]
[220. 233. 208. 237.]
[216. 208. 201. 226.]
[286. 237. 226. 319.]

rozkład Doolittla:
macierz L:
[1.0, 0, 0, 0]
[0.7801418439716312, 1.0, 0, 0]
[0.7659574468085106, 0.6434762510112102, 1.0, 0]
[1.0141843971631206, 0.2261643360684154, -0.1966843274653922, 1.0]
macierz U:
[282.0, 220.0, 216.0, 286.0]
[0, 61.36879432624113, 39.48936170212767, 13.879432624113463]
[0, 0, 10.142725066450907, -1.9949150583612436]
[0, 0, 0, 25.411861220304274]
wektor x:

**przetestowanie wydajnośći** - testuję wydajność mierząc czasy faktoryzacji LU dla każdej z 3 wcześniej zaimplementowanych metod, poza środowiskiem jupyter - wykres, oraz wyniki dołączyłem w pliku pdf
<br>do wykonania pomiarów wykorzystąłem poniższy kod:

In [0]:
import time
from math import sqrt
from random import randrange
import numpy as np
import matplotlib.pyplot as plt


def random_matrix(low, high, n):
    A = np.random.randint(low, high, size=(n, n))
    return np.dot(A, A.transpose()), np.random.randint(low, high, size=(n, 1))


def measure_time():
    n = randrange(20, 100)
    A, b = random_matrix(40, 160, n)

    start = time.time()
    crout(A)
    stop = time.time()
    crout_time = stop - start

    start = time.time()
    doolittle(A)
    stop = time.time()
    doolittle_time = stop - start

    start = time.time()
    cholesky(A)
    stop = time.time()
    cholesky_time = stop - start

    return crout_time, doolittle_time, cholesky_time, n


if __name__ == '__main__':
    crout_tab = []
    doolittle_tab = []
    cholesky_tab = []
    size = []
    for i in range(100):
        crout_time, doolittle_time, cholesky_time, n = measure_time()
        crout_tab.append(crout_time)
        doolittle_tab.append(doolittle_time)
        cholesky_tab.append(cholesky_time)
        size.append(n)

    for i in range(100):
        f = open("file.txt", "a")
        f.write(f"matrix size: {size[i]} \t crout: {crout_tab[i]}[s] \t doolittle: {doolittle_tab[i]}[s] \t cholesky: {cholesky_tab[i]}[s]"'\n')
        f.close()

    plt.plot(size, crout_tab, 'g.', label="crout method")
    plt.plot(size, doolittle_tab, 'r.', label="doolitle method")
    plt.plot(size, cholesky_tab, 'b.', label="cholesky method")
    plt.xlabel("matrix size")
    plt.ylabel("time[s]")
    plt.title("czas faktoryzacji LU")
    plt.legend()
    plt.grid(True)
    plt.savefig("LU")
    plt.show()

## Zadanie 3
Zapoznaj się z funkcją rozwiązywania układów równań liniowych dostarczoną przez bibliotekę numpy/scipy. Porównaj jej wydajność z własnymi implementacjami.

do wykonania pomiarów wykorzystałem poniższy kod:

In [0]:
import time
from math import sqrt
from random import randrange
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from scipy import linalg


def random_matrix(low, high, n):
    A = np.random.randint(low, high, size=(n, n))
    return np.dot(A, A.transpose()), np.random.randint(low, high, size=(n, 1))


def decompose_and_solve(A, Y, method):
    L, U = method(A)
    z = np.linalg.solve(L, Y)
    x = np.linalg.solve(U, z)
    return x


def measure_time():
    n = randrange(20, 100)
    A, b = random_matrix(40, 160, n)

    start = time.time()
    decompose_and_solve(A, b, crout)
    stop = time.time()
    crout_time = stop - start

    start = time.time()
    decompose_and_solve(A, b, doolittle)
    stop = time.time()
    doolittle_time = stop - start

    start = time.time()
    decompose_and_solve(A, b, cholesky)
    stop = time.time()
    cholesky_time = stop - start

    start = time.time()
    np.linalg.solve(A, b)
    stop = time.time()
    numpy_time = stop - start

    start = time.time()
    sc.linalg.solve(A, b)
    stop = time.time()
    scipy_time = stop - start

    return crout_time, doolittle_time, cholesky_time, numpy_time, scipy_time, n


if __name__ == '__main__':
    crout_tab = []
    doolittle_tab = []
    cholesky_tab = []
    numpy_tab = []
    scipy_tab = []
    size = []
    for i in range(100):
        crout_time, doolittle_time, cholesky_time, numpy_time, scipy_time, n = measure_time()
        crout_tab.append(crout_time)
        doolittle_tab.append(doolittle_time)
        cholesky_tab.append(cholesky_time)
        numpy_tab.append(numpy_time)
        scipy_tab.append(scipy_time)
        size.append(n)

    for i in range(100):
        f = open("file_2.txt", "a")
        f.write(f"matrix size: {size[i]} \t crout: {crout_tab[i]}[s] \t doolittle: {doolittle_tab[i]}[s] \t cholesky: {cholesky_tab[i]}[s] \t numpy: {numpy_tab[i]}[s] \t scipy: {scipy_tab[i]}[s]"'\n')
        f.close()

    plt.plot(size, crout_tab, 'g.', label="crout method")
    plt.plot(size, doolittle_tab, 'r.', label="doolitle method")
    plt.plot(size, cholesky_tab, 'b.', label="cholesky method")
    plt.plot(size, numpy_tab, 'c.', label="numpy method")
    plt.plot(size, scipy_tab, 'm.', label="scipy method")
    plt.xlabel("matrix size")
    plt.ylabel("time[s]")
    plt.title("czas faktoryzacji LU")
    plt.legend()
    plt.grid(True)
    plt.savefig("LU_2")
    plt.show()